In [2]:
#IMPORTING DEPENDENCIES

from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [3]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
instance = Browser("chrome", **executable_path, headless=False)